### Outline

0. Load Data and Restore results
<br />
<br />
1. Stabilization for linear transformations
<br />
<br />
2. 

SpaGFT is a python package to analyze spatial transcriptomics data via graph Fourier transform. To install SpaGFT, the python version is required to be >= 3.7. You can check your python version by:

In [1]:
import platform
platform.python_version()

'3.8.20'

In [2]:
# Import stages
from stage_0_setup import stage_0_main as s0
from stage_1_lin_trns import stage_1_main as s1

# Another imports
from contextlib import redirect_stdout
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


## Stage 0：Load Data and Restore results


During this step we will clone/load data, transform it to anndata object and run local SpaGFT to get SVG list. Then compare top-100 SVG list output with the SVG atlas at https://huggingface.co/spaces/jiawennnn/STimage-benchmark. We will measure the consistency by the Jaccard Index between those SVG lists.

This process is being done for 4 slides tested in the arxiv paper at https://arxiv.org/html/2510.07653v1#S12. 

In [3]:
tissue_dict = {
    'Human_Breast_Andersson_10142021_ST_A1': {'name': 'Human Breast Andersson', 'tech': 'ST'},
    'Human_Brain_Maynard_02082021_Visium_151676': {'name': 'Human Brain Maynard', 'tech': 'Visium'},
    'Human_Breast_10X_06092021_Visium': {'name': 'Human Breast 10X', 'tech': 'Visium'},
    'GSE213688_GSM6592060': {'name': 'GSE213688_GSM6592060', 'tech': 'Visium'}
}

In [4]:
# Initialization
stage_0_results_list = []

# Loop over tissues
for slide_id, slide_param in tissue_dict.items():    
    # Clone/Load data, run SpaGFT and compare top-100 SVG list with the SVG atlas
    slide_metrics = s0.run_stage_0_pipeline(
        slide_id=slide_id, tech=slide_param['tech'],
        restrict_logs=True
    )
    stage_0_results_list.append({
        'tissue': slide_param['name'], 'tech': slide_param['tech'], 
        'Jaccard': slide_metrics['jaccard']
    })

slide_metrics = []

In [5]:
# Show jaccard index for successfull restoring the atlas results
pd.DataFrame(stage_0_results_list)

,tissue,tech,Jaccard
0,Human Breast Andersson,ST,0.785714
1,Human Brain Maynard,Visium,0.923077
2,Human Breast 10X,Visium,0.960784
3,GSE213688_GSM6592060,Visium,0.818182


## Stage 1：Stability Check
Examine the Stability for linear transformations by 8 different transformation scenarios

### Run

In [ ]:
# Add tissues from ExSeq technology
tissue_dict['fem4_WT_F11'] = {'name': 'Mouse Brain WT fem4', 'tech': 'exseq'}
tissue_dict['fem3_5x_E7_A_left'] = {'name': 'Mouse Brain 5x fem3', 'tech': 'exseq'}

In [ ]:
stage_1_results = {}
# Loop over tissues
for slide_id, slide_param in tissue_dict.items():
    # Run stability test for the default scenarios
    stage_1_results[slide_param['name']] = s1.run_stability_test(
        slide_id=slide_id, tech=slide_param['tech'], 
        save_results=True, restrict_logs=True
    )

### Plot Stability Boxplot

In [ ]:
# For each tissue, show the distribution of Jaccard index across tissues
jaccard_records = []
for tissue, results in stage_1_results.items():
    for row in results.itertuples():
        jaccard_records.append({
            'tissue': tissue,
            'transformation': row.name,
            'Jaccard': row.Jaccard
        })
pd.DataFrame(jaccard_records)

# Plotting the Jaccard index distributions
plt.figure(figsize=(10, 6))
sns.boxplot(x='tissue', y='Jaccard', data=pd.DataFrame(jaccard_records), palette="Set3")
plt.ylim(0, 1.2)
plt.title('Jaccard Index Distribution Across Tissues')
plt.xlabel('Tissue')
plt.ylabel('Jaccard Index')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='x')

plt.show()

## Stage 2：Stability Improvement by Changing Graph Type
Benchmark Stability results under changing the graph representation of tissues before running SpaGFT